In [4]:
import os, sys, inspect
import pandas as pd
import numpy as np
from datetime import date
from collections import OrderedDict

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML, clear_output
import ipywidgets as widgets
from qgrid import show_grid

In [5]:
import src
from src.analysis.processing import Analysis

In [6]:
analysis = Analysis()

In [7]:
display(Markdown('<h2>Customer Segmentation Analysis</h2>'))
sections = ["Customer Data"]

sub_sections = ["Data Summary", "Missing Analysis", "Outlier Analysis", "Statistical Study"]

accordions = OrderedDict()
accordions["Loading"] = widgets.Accordion(children=[widgets.Output() for section in sections])
[accordions["Loading"].set_title(i, section) for i, section in enumerate(sections)]

for section in sections:
    accordions[section] = widgets.Accordion(children=[widgets.Output() for sub_section in sub_sections])
    [accordions[section].set_title(i, sub_section) for i, sub_section in enumerate(sub_sections)]
    
widget_fields = widgets.Tab(children=[accordions[t] for t in accordions])
[widget_fields.set_title(i, sub) for i, sub in enumerate(accordions.keys())]

<h2>Customer Segmentation Analysis</h2>

[None, None]

In [8]:
widget_fields

In [9]:
section = "Customer Data"

with accordions["Loading"].children[sections.index(section)]:
    clear_output()
    analysis.get_data()

In [7]:
print(analysis.data["customer_data"].shape)

analysis.data["customer_data"].head()

(32434489, 15)


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_name,aisle_id,product_group_id,aisle,product_group
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,eggs,dairy eggs
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,fresh vegetables,produce
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,spices seasonings,pantry
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,oils vinegars,pantry
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,baking ingredients,pantry


## Resources:
***
Customer segmentation learning on unsupervised learning from Kaggle challenge - <a href="https://www.kaggle.com/c/instacart-market-basket-analysis">Insta Cart Market Basket Analysis</a>

1. Features Explanation: [Information on each features]('https://www.lexjansen.com/sesug/2019/SESUG2019_Paper-252_Final_PDF.pdf')